In [1]:
import face_recognition
from face_recognition import face_locations
import os
import pandas as pd
import h5py
from tqdm import tqdm
from recursive_dict import *
import scipy.io
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import pickle


In [2]:

loaded_model = pickle.load(open('model.pkl', 'rb'))


In [3]:
loaded_model

MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=5000, verbose=True)

In [4]:
# set data paths
data_dir = '/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model'
img_dir = os.path.join(data_dir,'df_2')
indices_path = os.path.join(data_dir, 'indices_train_test.mat')
attr_path = os.path.join(data_dir, 'lfw_att_73.mat')

In [5]:

attr_name_mat = '/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/LWFA+/attrname.mat'
attr_name = scipy.io.loadmat(attr_name_mat)['AttrName']
attr_name = [str(s[0]) for s in attr_name.tolist()[0]]

In [6]:
FichList = [ f for f in os.listdir('/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/df_2') if os.path.isfile(os.path.join('/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/df_2',f)) ]
print(FichList)

['images.jpeg', 'IMG_7858.jpg', 'téléchargement.jpeg', 'téléchargement (5).jpeg', 'téléchargement (3).jpeg', 'téléchargement (1).jpeg', 'téléchargement (6).jpeg', 'téléchargement (4).jpeg', 'téléchargement (2).jpeg']


In [7]:
df_label = pd.DataFrame(columns=attr_name, index= FichList)

In [8]:
df_label

,Male,Asian,White,Black,Baby,Child,Youth,Middle Aged,Senior,Black Hair,...,Pale Skin,5 o Clock Shadow,Strong Nose-Mouth Lines,Wearing Lipstick,Flushed Face,High Cheekbones,Brown Eyes,Wearing Earrings,Wearing Necktie,Wearing Necklace
images.jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IMG_7858.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement.jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (5).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (3).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (1).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (6).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (4).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (2).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_label.index

Index(['images.jpeg', 'IMG_7858.jpg', 'téléchargement.jpeg',
       'téléchargement (5).jpeg', 'téléchargement (3).jpeg',
       'téléchargement (1).jpeg', 'téléchargement (6).jpeg',
       'téléchargement (4).jpeg', 'téléchargement (2).jpeg'],
      dtype='object')

In [10]:
vecs = []
fnames = []
i = 0
for fname in tqdm(df_label.index):
    i += 1
    img_path = os.path.join(img_dir, fname)
    X_img = face_recognition.load_image_file(img_path)
    X_faces_loc = face_recognition.face_locations(X_img)
    if len(X_faces_loc) != 1:
        continue
    faces_encoding = face_recognition.face_encodings(X_img, known_face_locations=X_faces_loc)[0]
    
    vecs.append(faces_encoding)
    fnames.append(fname)
    
df_feat = pd.DataFrame(vecs, index=fnames)
df_label = df_label[df_label.index.isin(df_feat.index)]
df_feat.sort_index(inplace=True)
df_label.sort_index(inplace=True)

100%|█████████████████████████████████████████████| 9/9 [00:01<00:00,  6.74it/s]
/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_72787/3413094446.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_label.sort_index(inplace=True)


from PIL import Image
import face_recognition

# Load the jpg file into a numpy array
img_path = os.path.join(img_dir, fname)
image = face_recognition.load_image_file(img_path)

# Find all the faces in the image using the default HOG-based model.
# This method is fairly accurate, but not as accurate as the CNN model and not GPU accelerated.
# See also: find_faces_in_picture_cnn.py
face_locations = face_recognition.face_locations(image)

print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

In [11]:
img_path

'/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/df_2/téléchargement (2).jpeg'

In [12]:
fnames

['images.jpeg',
 'IMG_7858.jpg',
 'téléchargement.jpeg',
 'téléchargement (3).jpeg',
 'téléchargement (6).jpeg',
 'téléchargement (4).jpeg',
 'téléchargement (2).jpeg']

In [13]:
loaded_model.predict(vecs)

/opt/anaconda3/envs/emlyon/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1,

In [14]:
predict_score = loaded_model.predict_proba(vecs)

/opt/anaconda3/envs/emlyon/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [15]:
df_2 = pd.DataFrame(predict_score, columns=df_label.columns)

In [16]:
df_2

,Male,Asian,White,Black,Baby,Child,Youth,Middle Aged,Senior,Black Hair,...,Pale Skin,5 o Clock Shadow,Strong Nose-Mouth Lines,Wearing Lipstick,Flushed Face,High Cheekbones,Brown Eyes,Wearing Earrings,Wearing Necktie,Wearing Necklace
0,0.071977,0.027395,0.029844,0.431434,0.000467,0.223442,0.733406,0.001580,0.001133,0.104179,...,0.143725,0.042219,0.494209,0.599698,0.055374,0.437564,0.890641,0.443413,0.063946,0.660777
1,0.055216,0.238553,0.053191,0.027869,0.001186,0.061919,0.862617,0.000272,0.001172,0.390138,...,0.456074,0.030817,0.758538,0.858977,0.117618,0.937497,0.941909,0.595744,0.065914,0.821428
2,0.095703,0.051153,0.965649,0.006452,0.001509,0.066321,0.858762,0.000844,0.005508,0.002446,...,0.673030,0.005500,0.065722,0.924641,0.063704,0.135273,0.131474,0.727642,0.142755,0.665726
3,0.034727,0.002825,0.990861,0.005139,0.000127,0.077490,0.793430,0.010132,0.003408,0.007951,...,0.878210,0.022837,0.034530,0.978536,0.009900,0.187859,0.050397,0.805389,0.057698,0.863229
4,0.043651,0.024511,0.788872,0.000431,0.001083,0.094003,0.844414,0.023716,0.000524,0.037148,...,0.756518,0.030649,0.317662,0.837471,0.021438,0.314718,0.571343,0.258168,0.069065,0.781885
5,0.126582,0.002684,0.922496,0.005586,0.008494,0.248239,0.569711,0.007248,0.003119,0.010449,...,0.576839,0.096676,0.170650,0.426619,0.026391,0.117746,0.469140,0.193167,0.088930,0.322579
6,0.023561,0.072655,0.517050,0.014811,0.000666,0.085208,0.884157,0.007788,0.000362,0.090727,...,0.630903,0.019745,0.376324,0.917373,0.018429,0.582845,0.638340,0.779135,0.035260,0.898553


In [17]:
df_2.to_csv("Résultats_test_lol.csv")